In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Create a dash application
app = dash.Dash(__name__)

# Download and read the `spacex_launch_geo.csv`
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.columns =[column.replace(" ", "_") for column in spacex_df.columns]
spacex_df.rename(columns = {'class':'Launch_Class'}, inplace = True)

max_payload = spacex_df['Payload_Mass_(kg)'].max()
min_payload = spacex_df['Payload_Mass_(kg)'].min()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Div(["Launch Site: ", dcc.Dropdown(id='site-dropdown', options=[
                                    {'label': 'All Sites', 'value': 'ALL'},
                                    {'label': 'CCAFS_LC_40', 'value':'CCAFS_LC_40'},
                                    {'label': 'CCAFS_SLC_40', 'value':'CCAFS_SLC_40'},
                                    {'label': 'KSC_LC_39A', 'value':'KSC_LC_39A'},
                                    {'label': 'VAFB_SLC_4E', 'value':'VAFB_SLC_4E'}], 
                                value = 'ALL', 
                                placeholder = "Select a Launch Site here", 
                                searchable = True)]), 
                                html.Br(),
                                
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the sitehtml.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),

# TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000, 
                                                         marks={0:{'label': '0 kg', 'style': {'color': '#f50'}}, 
                                                                2500:{'label': '2500 kg', 'style': {'color': '#f50'}}, 
                                                                5000:{'label': '5000 kg', 'style': {'color': '#f50'}}, 
                                                                7500:{'label': '7500 kg', 'style': {'color': '#f50'}}, 
                                                                10000:{'label': '10000 kg', 'style': {'color': '#f50'}}}, 
                                                         value=[min_payload, max_payload])), 
                                                                                                                        

# TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')), ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='Launch_Class', 
        names= 'Launch_Site', 
        title='Success and Launch Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch_Site'] == (entered_site)]
        filtered_df = filtered_df.groupby(['Launch_Site', 'Launch_Class']).size().reset_index(name ='Launch_Class count')
        fig = px.pie(filtered_df, values = 'Launch_Class count', names = 'Launch_Class', title= f'Total Success Launches for {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback([Output(component_id='success-payload-scatter-chart', component_property='figure')],
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, entered_payload):
    filtered_df = spacex_df
    if entered_site == 'ALL': 
        fig2 = px.scatter(spacex_df, y = 'Launch_Class', x = 'Payload_Mass_(kg)', color = 'Booster_Version_Category', 
        title = 'Correlation between Payload and Success for all Sites')
        return fig2.show()
    else: 
        filtered_df = spacex_df[spacex_df['Launch_Site'] == (entered_site) & spacex_df['Payload_Mass_(kg)' == (entered_payload)]]
        fig2 = px.scatter(filtered_df, x = 'Payload_Mass_(kg)', y = 'Launch_Class', color = 'Booster_Version_Category', 
        title = f'Correlation between Payload and Success for {entered_site}')
        return fig2.show()

# Run the app
if __name__ == '__main__':
    app.run_server()

